In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.cluster import KMeans
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
df=pd.read_excel('Online Retail.xlsx')

In [5]:
df=df[~(df.CustomerID.isnull())]
df = df[~(df.Quantity<0)]
df = df[df.UnitPrice>0]
df.reset_index(inplace=True)

In [6]:
x=df['StockCode'].value_counts()
item_type_mapping={}
item_list=x.index
for i in range(0,len(item_list)):
    item_type_mapping[item_list[i]]=i

df['StockCode']=df['StockCode'].map(lambda x:item_type_mapping[x]) 

In [7]:
df['amount'] = df.Quantity*df.UnitPrice
df['amount']

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
397879    10.20
397880    12.60
397881    16.60
397882    16.60
397883    14.85
Name: amount, Length: 397884, dtype: float64

In [8]:
df = df[~(df.Quantity>df.Quantity.quantile(0.95))]
df = df[~(df.UnitPrice>df.UnitPrice.quantile(0.95))]

In [9]:
df['year'] = df['InvoiceDate'].dt.year
df['month'] = df['InvoiceDate'].dt.month
df['hour'] = df['InvoiceDate'].dt.hour
df['weekday'] = df['InvoiceDate'].dt.day_name()

In [10]:
df.drop('index', axis=1, inplace=True)

In [11]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount,year,month,hour,weekday
0,536365,0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010,12,8,Wednesday
1,536365,407,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,8,Wednesday
2,536365,458,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010,12,8,Wednesday
3,536365,280,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,8,Wednesday
4,536365,276,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,8,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397879,581587,1064,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,2011,12,12,Friday
397880,581587,411,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,2011,12,12,Friday
397881,581587,364,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,2011,12,12,Friday
397882,581587,866,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,2011,12,12,Friday


In [12]:
dfv=df.copy()

#**Apriori Algorithm**

In [13]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [14]:
dfv

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount,year,month,hour,weekday
0,536365,0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010,12,8,Wednesday
1,536365,407,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,8,Wednesday
2,536365,458,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010,12,8,Wednesday
3,536365,280,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,8,Wednesday
4,536365,276,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,8,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397879,581587,1064,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,2011,12,12,Friday
397880,581587,411,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,2011,12,12,Friday
397881,581587,364,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,2011,12,12,Friday
397882,581587,866,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,2011,12,12,Friday


here we are making the dataset suitable for apriori algorithm

In [15]:
basket = (dfv.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
basket

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,200 RED + WHITE BENDY STRAWS,3 BIRDS CANVAS SCREEN,3 BLACK CATS W HEARTS BLANK CARD,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 GARDENIA MORRIS BOXED CANDLES,3 HEARTS HANGING DECORATION RUSTIC,...,YELLOW COAT RACK PARIS FASHION,YELLOW DRAGONFLY HELICOPTER,YELLOW EASTER EGG HUNT START POST,YELLOW FELT HANGING HEART W FLOWER,YELLOW FLOWERS FELT HANDBAG KIT,YELLOW GIANT GARDEN THERMOMETER,YELLOW METAL CHICKEN HEART,YELLOW PINK FLOWER DESIGN BIG BOWL,YELLOW POT PLANT CANDLE,YELLOW SHARK HELICOPTER,YELLOW/BLUE RETRO RADIO,YELLOW/ORANGE FLOWER DESIGN PLATE,YELLOW/PINK FLOWER DESIGN BIG MUG,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,YULETIDE IMAGES S/6 PAPER BOXES,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

using the encode function to convert all int values greater than 0 to 1

In [16]:
basket_sets = basket.applymap(encode_units)

In [17]:
basket_sets

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,200 RED + WHITE BENDY STRAWS,3 BIRDS CANVAS SCREEN,3 BLACK CATS W HEARTS BLANK CARD,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 GARDENIA MORRIS BOXED CANDLES,3 HEARTS HANGING DECORATION RUSTIC,...,YELLOW COAT RACK PARIS FASHION,YELLOW DRAGONFLY HELICOPTER,YELLOW EASTER EGG HUNT START POST,YELLOW FELT HANGING HEART W FLOWER,YELLOW FLOWERS FELT HANDBAG KIT,YELLOW GIANT GARDEN THERMOMETER,YELLOW METAL CHICKEN HEART,YELLOW PINK FLOWER DESIGN BIG BOWL,YELLOW POT PLANT CANDLE,YELLOW SHARK HELICOPTER,YELLOW/BLUE RETRO RADIO,YELLOW/ORANGE FLOWER DESIGN PLATE,YELLOW/PINK FLOWER DESIGN BIG MUG,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,YULETIDE IMAGES S/6 PAPER BOXES,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
581585,0,0,0,0,0,0,

getting the frequent itemsets

In [18]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

applying association rules 

In [19]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.6)
rules.sort_values('consequent support', ascending=False).reset_index()

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,478,(NATURAL SLATE HEART CHALKBOARD ),(WHITE HANGING HEART T-LIGHT HOLDER),0.054341,0.108682,0.015101,0.277897,2.556980,0.009195,1.234337
1,474,(LUNCH BAG SUKI DESIGN ),(WHITE HANGING HEART T-LIGHT HOLDER),0.050784,0.108682,0.010145,0.199770,1.838123,0.004626,1.113828
2,38,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.071308,0.108682,0.013993,0.196239,1.805628,0.006244,1.108934
3,336,(LOVE BUILDING BLOCK WORD),(WHITE HANGING HEART T-LIGHT HOLDER),0.032301,0.108682,0.010029,0.310469,2.856684,0.006518,1.292645
4,57,(CANDLEHOLDER PINK HANGING HEART),(WHITE HANGING HEART T-LIGHT HOLDER),0.019882,0.108682,0.014285,0.718475,6.610819,0.012124,3.166037
...,...,...,...,...,...,...,...,...,...,...
935,919,(POPPY'S PLAYHOUSE KITCHEN),"(POPPY'S PLAYHOUSE LIVINGROOM , POPPY'S PLAYHO...",0.019824,0.011778,0.010495,0.529412,44.950204,0.010262,2.099972
936,544,(REGENCY TEA PLATE ROSES ),(REGENCY TEA PLATE PINK),0.016792,0.011661,0.010029,0.597222,51.214792,0.009833,2.453807
937,540,(REGENCY TEA PLATE GREEN ),(REGENCY TEA PLATE PINK),0.013993,0.011661,0.010378,0.741667,63.601625,0.010215,3.825828
938,152,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010670,0.010903,0.010087,0.945355,86.704743,0.009971,18.100472


when we used whole dataset, ignoring the countries, the support values are quite low, lets check if countrywise the support values vary by a lot

In [20]:
top_5=dfv.groupby('Country')['Quantity'].sum()

In [21]:
top_5.sort_values(ascending=False).head()

Country
United Kingdom    2377963
Germany             90958
France              81761
EIRE                65079
Switzerland         20847
Name: Quantity, dtype: int64

a function to create encoded dataset suitable to apriori

In [22]:
def create_basket(country_filter):
    basket = (dfv[dfv['Country'] == country_filter].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
    basket_sets = basket.applymap(encode_units)
    return basket_sets

In [23]:
france=create_basket('France')
f_frequent_itemsets = apriori(france, min_support=0.6, use_colnames=True)
f_rules=association_rules(frequent_itemsets, metric='lift', min_threshold=0.60).sort_values('consequent support', ascending=False)
f_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
478,(NATURAL SLATE HEART CHALKBOARD ),(WHITE HANGING HEART T-LIGHT HOLDER),0.054341,0.108682,0.015101,0.277897,2.556980,0.009195,1.234337
474,(LUNCH BAG SUKI DESIGN ),(WHITE HANGING HEART T-LIGHT HOLDER),0.050784,0.108682,0.010145,0.199770,1.838123,0.004626,1.113828
38,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.071308,0.108682,0.013993,0.196239,1.805628,0.006244,1.108934
336,(LOVE BUILDING BLOCK WORD),(WHITE HANGING HEART T-LIGHT HOLDER),0.032301,0.108682,0.010029,0.310469,2.856684,0.006518,1.292645
57,(CANDLEHOLDER PINK HANGING HEART),(WHITE HANGING HEART T-LIGHT HOLDER),0.019882,0.108682,0.014285,0.718475,6.610819,0.012124,3.166037
...,...,...,...,...,...,...,...,...,...
919,(POPPY'S PLAYHOUSE KITCHEN),"(POPPY'S PLAYHOUSE LIVINGROOM , POPPY'S PLAYHO...",0.019824,0.011778,0.010495,0.529412,44.950204,0.010262,2.099972
544,(REGENCY TEA PLATE ROSES ),(REGENCY TEA PLATE PINK),0.016792,0.011661,0.010029,0.597222,51.214792,0.009833,2.453807
540,(REGENCY TEA PLATE GREEN ),(REGENCY TEA PLATE PINK),0.013993,0.011661,0.010378,0.741667,63.601625,0.010215,3.825828
152,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010670,0.010903,0.010087,0.945355,86.704743,0.009971,18.100472


In [24]:
germany=create_basket('Germany')
g_frequent_itemsets = apriori(germany, min_support=0.6, use_colnames=True)
g_rules=association_rules(frequent_itemsets, metric='lift', min_threshold=0.60).sort_values('consequent support', ascending=False)
g_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
478,(NATURAL SLATE HEART CHALKBOARD ),(WHITE HANGING HEART T-LIGHT HOLDER),0.054341,0.108682,0.015101,0.277897,2.556980,0.009195,1.234337
474,(LUNCH BAG SUKI DESIGN ),(WHITE HANGING HEART T-LIGHT HOLDER),0.050784,0.108682,0.010145,0.199770,1.838123,0.004626,1.113828
38,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.071308,0.108682,0.013993,0.196239,1.805628,0.006244,1.108934
336,(LOVE BUILDING BLOCK WORD),(WHITE HANGING HEART T-LIGHT HOLDER),0.032301,0.108682,0.010029,0.310469,2.856684,0.006518,1.292645
57,(CANDLEHOLDER PINK HANGING HEART),(WHITE HANGING HEART T-LIGHT HOLDER),0.019882,0.108682,0.014285,0.718475,6.610819,0.012124,3.166037
...,...,...,...,...,...,...,...,...,...
919,(POPPY'S PLAYHOUSE KITCHEN),"(POPPY'S PLAYHOUSE LIVINGROOM , POPPY'S PLAYHO...",0.019824,0.011778,0.010495,0.529412,44.950204,0.010262,2.099972
544,(REGENCY TEA PLATE ROSES ),(REGENCY TEA PLATE PINK),0.016792,0.011661,0.010029,0.597222,51.214792,0.009833,2.453807
540,(REGENCY TEA PLATE GREEN ),(REGENCY TEA PLATE PINK),0.013993,0.011661,0.010378,0.741667,63.601625,0.010215,3.825828
152,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010670,0.010903,0.010087,0.945355,86.704743,0.009971,18.100472


In [25]:
uk=create_basket('United Kingdom ')
uk_frequent_itemsets = apriori(uk, min_support=0.6, use_colnames=True)
uk_rules=association_rules(frequent_itemsets, metric='lift', min_threshold=0.60).sort_values('consequent support', ascending=False)
uk_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
478,(NATURAL SLATE HEART CHALKBOARD ),(WHITE HANGING HEART T-LIGHT HOLDER),0.054341,0.108682,0.015101,0.277897,2.556980,0.009195,1.234337
474,(LUNCH BAG SUKI DESIGN ),(WHITE HANGING HEART T-LIGHT HOLDER),0.050784,0.108682,0.010145,0.199770,1.838123,0.004626,1.113828
38,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.071308,0.108682,0.013993,0.196239,1.805628,0.006244,1.108934
336,(LOVE BUILDING BLOCK WORD),(WHITE HANGING HEART T-LIGHT HOLDER),0.032301,0.108682,0.010029,0.310469,2.856684,0.006518,1.292645
57,(CANDLEHOLDER PINK HANGING HEART),(WHITE HANGING HEART T-LIGHT HOLDER),0.019882,0.108682,0.014285,0.718475,6.610819,0.012124,3.166037
...,...,...,...,...,...,...,...,...,...
919,(POPPY'S PLAYHOUSE KITCHEN),"(POPPY'S PLAYHOUSE LIVINGROOM , POPPY'S PLAYHO...",0.019824,0.011778,0.010495,0.529412,44.950204,0.010262,2.099972
544,(REGENCY TEA PLATE ROSES ),(REGENCY TEA PLATE PINK),0.016792,0.011661,0.010029,0.597222,51.214792,0.009833,2.453807
540,(REGENCY TEA PLATE GREEN ),(REGENCY TEA PLATE PINK),0.013993,0.011661,0.010378,0.741667,63.601625,0.010215,3.825828
152,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010670,0.010903,0.010087,0.945355,86.704743,0.009971,18.100472


In [26]:
s=create_basket('Switzerland')
s_frequent_itemsets = apriori(s, min_support=0.6, use_colnames=True)
s_rules=association_rules(frequent_itemsets, metric='lift', min_threshold=0.60).sort_values('consequent support', ascending=False)
s_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
478,(NATURAL SLATE HEART CHALKBOARD ),(WHITE HANGING HEART T-LIGHT HOLDER),0.054341,0.108682,0.015101,0.277897,2.556980,0.009195,1.234337
474,(LUNCH BAG SUKI DESIGN ),(WHITE HANGING HEART T-LIGHT HOLDER),0.050784,0.108682,0.010145,0.199770,1.838123,0.004626,1.113828
38,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.071308,0.108682,0.013993,0.196239,1.805628,0.006244,1.108934
336,(LOVE BUILDING BLOCK WORD),(WHITE HANGING HEART T-LIGHT HOLDER),0.032301,0.108682,0.010029,0.310469,2.856684,0.006518,1.292645
57,(CANDLEHOLDER PINK HANGING HEART),(WHITE HANGING HEART T-LIGHT HOLDER),0.019882,0.108682,0.014285,0.718475,6.610819,0.012124,3.166037
...,...,...,...,...,...,...,...,...,...
919,(POPPY'S PLAYHOUSE KITCHEN),"(POPPY'S PLAYHOUSE LIVINGROOM , POPPY'S PLAYHO...",0.019824,0.011778,0.010495,0.529412,44.950204,0.010262,2.099972
544,(REGENCY TEA PLATE ROSES ),(REGENCY TEA PLATE PINK),0.016792,0.011661,0.010029,0.597222,51.214792,0.009833,2.453807
540,(REGENCY TEA PLATE GREEN ),(REGENCY TEA PLATE PINK),0.013993,0.011661,0.010378,0.741667,63.601625,0.010215,3.825828
152,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010670,0.010903,0.010087,0.945355,86.704743,0.009971,18.100472


In [27]:
eire=create_basket('EIRE')
eire_frequent_itemsets = apriori(eire, min_support=0.6, use_colnames=True)
eire_rules=association_rules(frequent_itemsets, metric='lift', min_threshold=0.60).sort_values('consequent support', ascending=False)
eire_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
478,(NATURAL SLATE HEART CHALKBOARD ),(WHITE HANGING HEART T-LIGHT HOLDER),0.054341,0.108682,0.015101,0.277897,2.556980,0.009195,1.234337
474,(LUNCH BAG SUKI DESIGN ),(WHITE HANGING HEART T-LIGHT HOLDER),0.050784,0.108682,0.010145,0.199770,1.838123,0.004626,1.113828
38,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.071308,0.108682,0.013993,0.196239,1.805628,0.006244,1.108934
336,(LOVE BUILDING BLOCK WORD),(WHITE HANGING HEART T-LIGHT HOLDER),0.032301,0.108682,0.010029,0.310469,2.856684,0.006518,1.292645
57,(CANDLEHOLDER PINK HANGING HEART),(WHITE HANGING HEART T-LIGHT HOLDER),0.019882,0.108682,0.014285,0.718475,6.610819,0.012124,3.166037
...,...,...,...,...,...,...,...,...,...
919,(POPPY'S PLAYHOUSE KITCHEN),"(POPPY'S PLAYHOUSE LIVINGROOM , POPPY'S PLAYHO...",0.019824,0.011778,0.010495,0.529412,44.950204,0.010262,2.099972
544,(REGENCY TEA PLATE ROSES ),(REGENCY TEA PLATE PINK),0.016792,0.011661,0.010029,0.597222,51.214792,0.009833,2.453807
540,(REGENCY TEA PLATE GREEN ),(REGENCY TEA PLATE PINK),0.013993,0.011661,0.010378,0.741667,63.601625,0.010215,3.825828
152,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010670,0.010903,0.010087,0.945355,86.704743,0.009971,18.100472
